In [ ]:
import numpy as np
import pandas as pd
from pymystem3 import Mystem
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance as ds
import torch
import fasttext.util
import matplotlib
import matplotlib.pyplot as plt
import copy
import os
from scipy.special import expit
import lightgbm as gbm
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi, BM25Plus, BM25L
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import wget
import gensim
import zipfile
import fasttext.util
from sentence_transformers import SentenceTransformer, models
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [ ]:
# извлекаем и лемматизируем тайтлы и запросы (запросы прогнал через спеллчекер яндекса, а также и тайтлы и запросы через Яндекс Переводчик, чтобы перевести на англ)

def stem(doc):
  txt = doc.split('\t')[1].strip()
  qid = doc.split('\t')[0]
  st = Mystem()
  s = st.lemmatize(txt.lower())
  return qid, ''.join(s)

queries = {}
non_norm_q = {}
with open("Yandex_spell.txt", 'r') as f:
  txt = f.readlines()
  for i, qid, text in enumerate(list(map(stem, txt))):
    queries[int(qid)] = text.strip()
    non_norm_q[int(qid)] = txt[i].split('\t')[1].strip()

titles = []
dids = []
non_norm_t = {}
with open("titles.txt", 'r') as f:
  titles = f.readlines()
  for i, did, txt in enumerate(list(map(stem, titles))):
    titles.append(txt.strip())
    dids.append(int(did))
    non_norm_t[int(did)] = txt[i].split('\t')[1].strip()
  f.close()

en_queries = {}
en_titles = {}
en_titles_arr = []
en_dids = []
with open("Yandex_translate_queries.txt" ,'r') as f:
  for line in f.readlines():
    txt = line.split('\t')
    en_queries[int(txt[0])]= txt[1].lower().strip()
  f.close()
with open("Yandex_translate_titles.txt" ,'r') as f:
  for line in f.readlines():
    txt = line.split('\t')
    en_titles_arr.append(txt[1].lower().strip())
    en_dids.append(int(txt[0]))
    en_titles[int(txt[0])]= txt[1].lower().strip()
  f.close()

train_data = []
qmap = {}
f = open("train.marks.tsv", 'r')
for line in f.readlines():
    arr = line.split('\t')
    if int(arr[2]) == 0:
        continue
    train_data.append([int(arr[0]), int(arr[1]), int(arr[2])])
    if not (int(arr[0]) in qmap.keys()):
        qmap[int(arr[0])]=[int(arr[1])]
    else:
        qmap[int(arr[0])].append(int(arr[1]))
train_data = np.array(train_data)
f.close()

f = open("sample.csv", 'r')
f.readline()
test_data = []
for line in f.readlines():
    arr = line.split(',')
    test_data.append([int(arr[0]), int(arr[1]), -1])
    if int(arr[0]) not in qmap.keys():
        qmap[int(arr[0])]=[int(arr[1])]
    else:
        qmap[int(arr[0])].append(int(arr[1]))
test_data = np.array(test_data)
f.close()

In [ ]:
#каждому хосту ставим в соответствие hid
hmap = {}
with open("url.data", 'r') as f:
  for line in f.readlines():
    arr = line.split('\t')
    host = arr[1].split("/")[0]
    if host.startswith("www."):
      host = host[4:] 
    did = int(arr[0])
    hmap[host] = did

#словарь из страниц для каждого хоста
hosts = {}
with open("url.data", 'r') as f:
  for line in f.readlines():
    arr = line.split('\t')
    host = arr[1].split("/")[0]
    if host.startswith("www."):
      host = host[4:]
    hid = hmap[host]    
    did = int(arr[0])
    if hid in hosts.keys():
      hosts[hid].append(did)
    else:
      hosts[hid] = [did]

#поведенческие фичи
def get_beh_features(dir, query, host):
    Pages = {}
    for i in range(10):
      with open("click_data/"+dir+"/part-r-000"+str(i).zfill(2), 'r') as f:
        for line in f.readlines():
          data = {}
          line = line.strip().split('\t')
          if query:
              beg = 2
              end = 11
          else:
              beg = 1
              end = 10
          for k in range(beg, end):
              arr = line[k].strip().split(":")
              data[arr[0]] = float(arr[1])
          for k in range(end, end + 2):
            arr = line[k].strip().split(":")
            pos = []
            for ps in enumerate(arr[1].strip().split(" ")):
                pos.append(int(ps))
            data[arr[0]] = np.array(pos)
          if query:
                Pages[line[0]+','+line[1]] = data
          else:
                Pages[int(line[0])] = data
    
    features = {}
    gamma = np.array([0.41, 0.16, 0.105, 0.08, 0.06])
    for i in Pages.keys():
        ctr = Pages[i]['clicks'] / Pages[i]['shows'] #отношение числа кликов к числу показов
        first_ctr = Pages[i]['first_clicks'] / Pages[i]['shows'] #отношение первых кликов к числу показов
        last_ctr = Pages[i]['last_clicks'] / Pages[i]['shows'] # отношение последних кликов к числу показов
        mean_time = Pages[i]['time'] / Pages[i]['clicks'] #среднее время проведенное на странице
        pos_ctr = Pages[i]['pos_clicks'] / Pages[i]['pos_shows'] #отношение числа кликов к числу показов для каждой из топ 5 позиций выдачи
        not_top_shows = Docs[i]['shows_not_top'] / Docs[i]['shows'] #процент показов вне топ 5
        not_top_clicks = Docs[i]['clicks_not_top'] / Docs[i]['clicks'] #процент кликов вне топ 5
        not_top_ctr = not_top_clicks / not_top_shows #CTR вне топа
        pos_click_probability = Pages[i]['pos_clicks'] / Pages[i]['clicks'] #проценты кликов от общего числа в топ 5
        PBM = Docs[i]['clicks'] / np.sum(Docs[i]['shows_pos'] * gamma) #PBM
        mean_after_clicks = 0
        mean_before_clicks = 0
        if Pages[i]['clicks'] - Pages[i]['first_clicks'] != 0:
            mean_before_clicks = Pages[i]['before_clicks'] / (Pages[i]['clicks'] - Pages[i]['first_clicks']) #среднее кол-во кликов до перехода к тек. странице
        if Pages[i]['clicks'] - Pages[i]['last_clicks'] != 0:
            mean_after_clicks = Pages[i]['after_clicks'] / (Pages[i]['clicks'] - Pages[i]['last_clicks']) #среднее кол-во кликов после перехода к тек. странице
        f = [ctr, first_ctr, last_ctr, mean_time, pos_ctr, not_top_shows, not_top_clicks, not_top_ctr, pos_click_probability, mean_before_clicks, mean_after_clicks, PBM]
        if not host:
            features[i] = f   
        if host and query:
            s = i.split(',')
            if int(s[1]) in hosts.keys():
                for doc in hosts[int(s[1])]:
                    if doc in qmap[int(s[0])]:
                        features[s[0]+","+ str(doc)] = f
        if host and not query:
            if i in hosts.keys():
                for doc in hosts[i]:
                    features[doc] = f
    return features

docs_features = get_beh_features("docs", False, False)
hosts_features = get_beh_features("hosts", False, True)
qd_features= get_beh_features("query-docs", True, False)
qh_features = get_beh_features("query-hosts", True, True)

In [ ]:
stop = get_stop_words('en') + get_stop_words('ru') + ['aren', 'can', 'isn', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven']

#преобразуем текст в нграммы по словам или все сразу
def parse_ngrams(text, n = 1, word = True):
    if word:
        n_grams = ngrams(word_tokenize(text), n)
        return [' '.join(grams) for grams in n_grams]
    else:
        n_grams = ngrams(text, n)
        return [''.join(grams) for grams in n_grams]

#генератор возварщающий тайтлы документов по doc_ids на английском или русском
def gen_titles(dids, bm25 = False, n = 1, word = True, english = False):
    for i in dids:
        if bm25:
          if english:
            yield parse_ngrams(en_titles[i], n=n, word=word)
          else:
            yield parse_ngrams(titles[i], n=n, word=word)
        else:
            yield titles[i]

#разделил доки по запросам, это генератор, возвращающий тексты доков 
def gen_docs_qid(qid):
    docs = {}
    f = open("documents_by_qid/{}.txt".format(qid), 'r')
    for line in f.readlines():
        arr = line.split('\t')
        docs[int(arr[0])] = arr[1].strip()
    f.close()
    for i in qmap[qid]:
        if i in doc.keys():
            yield docs[i]
    del docs

In [ ]:
#фичи BM25 и TF * IDF на тайтлах
tfidfbm25_features = {}
for i in range(len(qmap)):
    vectorizer = TfidfVectorizer(use_idf = False)
    D = vectorizer.fit_transform(gen_titles(qmap[i]))
    q = vectorizer.transform([queries[i]])

    f1 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f2 = cosine_similarity(D, q).flatten()

    vectorizer = TfidfVectorizer(ngram_range=(2,2), stop_words=stop, use_idf = False)
    D=vectorizer.fit_transform(gen_titles(qmap[i]))
    q=vectorizer.transform([queries[i]])

    f3 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f4 = cosine_similarity(D,q).flatten()

    vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=stop, use_idf = False)
    D = vectorizer.fit_transform(gen_titles(qmap[i]))
    q = vectorizer.transform([queries[i]])

    f5 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f6 = cosine_similarity(D,q).flatten()

    vectorizer = TfidfVectorizer(ngram_range=(3,9), stop_words=stop, analyzer='char_wb', use_idf = False)
    D = vectorizer.fit_transform(gen_titles(qmap[i]))
    q = vectorizer.transform([queries[i]])

    f7 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f8 = cosine_similarity(D,q).flatten()

    vectorizer = TfidfVectorizer()
    D = vectorizer.fit_transform(gen_titles(qmap[i]))
    q = vectorizer.transform([queries[i]])

    f9 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f10 = cosine_similarity(D, q).flatten()

    vectorizer = TfidfVectorizer(ngram_range=(2,2), stop_words=stop)
    D=vectorizer.fit_transform(gen_titles(qmap[i]))
    q=vectorizer.transform([queries[i]])

    f11 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f12 = cosine_similarity(D,q).flatten()

    vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=stop)
    D = vectorizer.fit_transform(gen_titles(qmap[i]))
    q = vectorizer.transform([queries[i]])

    f13 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f14 = cosine_similarity(D,q).flatten()

    vectorizer = TfidfVectorizer(ngram_range=(3,9), stop_words=stop, analyzer='char_wb')
    D = vectorizer.fit_transform(gen_titles(qmap[i]))
    q = vectorizer.transform([queries[i]])

    f15 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])
    f16 = cosine_similarity(D,q).flatten()
    
    
      
    
    grammas = list(gen_titles(qmap[i], bm25 = True, n = 1))
    bm25 = BM25Plus(grammas)
    f17 = bm25.get_scores(parse_ngrams(queries[i], n = 1, word = True))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 2))
    bm25 = BM25Plus(grammas)
    f18 = bm25.get_scores(parse_ngrams(queries[i], n = 2, word = True))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 2, word = False))
    bm25 = BM25Plus(grammas)
    f19 = bm25.get_scores(parse_ngrams(queries[i], n = 2, word = False))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 3, word = False))
    bm25 = BM25Plus(grammas)
    f20 = bm25.get_scores(parse_ngrams(queries[i], n = 3, word = False))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 5, word = False))
    bm25 = BM25Plus(grammas)
    f21 = bm25.get_scores(parse_ngrams(queries[i], n = 5, word = False))
    
    grammas = list(gen_titles(qmap[i], bm25 = True, n = 1))
    bm25 = BM25L(grammas)
    f22 = bm25.get_scores(parse_ngrams(queries[i], n = 1, word = True))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 2))
    bm25 = BM25L(grammas)
    f23 = bm25.get_scores(parse_ngrams(queries[i], n = 2, word = True))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 2, word = False))
    bm25 = BM25L(grammas)
    f24 = bm25.get_scores(parse_ngrams(queries[i], n = 2, word = False))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 3, word = False))
    bm25 = BM25L(grammas)
    f25 = bm25.get_scores(parse_ngrams(queries[i], n = 3, word = False))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 5, word = False))
    bm25 = BM25L(grammas)
    f26 = bm25.get_scores(parse_ngrams(queries[i], n = 5, word = False))


    grammas = list(gen_titles(qmap[i], bm25 = True, n = 1))
    bm25 = BM25Okapi(grammas)
    f27 = bm25.get_scores(parse_ngrams(queries[i], n = 1, word = True))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 2))
    bm25 = BM25Okapi(grammas)
    f28 = bm25.get_scores(parse_ngrams(queries[i], n = 2, word = True))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 2, word = False))
    bm25 = BM25Okapi(grammas)
    f29 = bm25.get_scores(parse_ngrams(queries[i], n = 2, word = False))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 3, word = False))
    bm25 = BM25Okapi(grammas)
    f30 = bm25.get_scores(parse_ngrams(queries[i], n = 3, word = False))

    grammas = list(gen_titles(qmap[i], bm25 = True, n = 5, word = False))
    bm25 = BM25Okapi(grammas)
    f31 = bm25.get_scores(parse_ngrams(queries[i], n = 5, word = False))
    
    
    
    
    for k, j in enumerate(qmap[i]):
        tfidfbm25_features[str(i) + "," + str(j)] = [f1[k], f2[k], f3[k], f4[k], f5[k], f6[k], f7[k], f8[k], f9[k], f10[k], f11[k], f12[k], f13[k], f14[k], f15[k], f16[k], f17[k], f18[k], f19[k], f20[k], f21[k], f22[k], f23[k], f24[k], f25[k], f26[k], f27[k], f28[k], f29[k], f30[k], f31[k]]

In [ ]:
#фичи BM25 и TF * IDF на текстах доков, особо много фичей не сделаешь - долго считает
def tfidfbm25_doc(qid):
    vectorizer = TfidfVectorizer(ngram_range = (1, 3), stop_words = stop)
    D = vectorizer.fit_transform(gen_docs_qid(qid))
    q = vectorizer.transform([queries[qid]])
    f1 = cosine_similarity(D, q).flatten()
    f2 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])

    vectorizer = TfidfVectorizer(ngram_range = (1, 1), stop_words = stop)
    D = vectorizer.fit_transform(gen_docs_qid(qid))
    q = vectorizer.transform([queries[qid]])
    f3 = cosine_similarity(D, q).flatten()
    f4 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])

    vectorizer = TfidfVectorizer(ngram_range = (2, 2), stop_words = stop)
    D = vectorizer.fit_transform(gen_docs_qid(qid))
    q = vectorizer.transform([queries[qid]])
    f5 = cosine_similarity(D, q).flatten()
    f6 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])

    vectorizer = TfidfVectorizer(ngram_range = (3, 9), stop_words = stop, analyzer = 'char_wb')
    D = vectorizer.fit_transform(gen_docs_qid(qid))
    q = vectorizer.transform([queries[qid]])
    f7 = cosine_similarity(D,q).flatten()
    f8 = (((D.dot(q.T)).toarray()).T).reshape(D.shape[0])

    grammas = list(gen_docs_qid(qid))
    bm25 = BM25Okapi(grammas)
    f9 = bm25.get_scores(parse_ngrams(queries[i], n = 1, word = True))

    grammas = list(gen_docs_qid(qid))
    bm25 = BM25Okapi(grammas)
    f10 = bm25.get_scores(parse_ngrams(queries[i], n = 2, word = True))

    return qid, np.vstack((f1, f2, f3, f4, f5, f6, f7, f8, f9, f10))


doc_tdidfbm25_features={}
for i, f in list(map(tfidfbm25_doc, range(6311))):
    for k, j in zip(qmap[i], f):
        doc_tdidfbm25_features[str(i) + "," + str(k)] = j

In [ ]:
#модели фасттекст
en_model = fasttext.load_model('cc.en.300.bin')
ru_model = fasttext.load_model('cc.ru.300.bin')
ru_model_news = fasttext.load_model('ft_native_300_ru_wiki_lenta_lemmatize.bin') #модель из системы рекомендации новостей https://github.com/temur-kh/news-recommendation-system


#фичи фасттекст
def features_ft(qid):
    features = []
    v1_q = np.zeros(300)
    v2_q = np.zeros(300)
    v3_q = np.zeros(300)
    v4_q = ru_model.get_sentence_vector(queries[qid])
    v5_q = en_model.get_sentence_vector(queries[qid])
    v6_q = ru_model_news.get_sentence_vector(queries[qid])
    for word in queries[qid].split():
        v1_q += ru_model.get_word_vector(word)
        v2_q += en_model.get_word_vector(word)
        v3_q += ru_model_news.get_word_vector(word)
    for title in list(gen_titles(qmap[qid])):
        v1_d = np.zeros(300)
        v2_d = np.zeros(300)
        v3_d = np.zeros(300)
        v4_d = ru_model.get_sentence_vector(title)
        v5_d = en_model.get_sentence_vector(title)
        v6_d = ru_model_news.get_sentence_vector(title)
        for word in title.split():
            v1_d += ru_model.get_word_vector(word)
            v2_d += en_model.get_word_vector(word)
            v3_d += ru_model_news.get_word_vector(word)
        feaatures.append([np.dot(v1_q,v1_d), np.dot(v2_q,v2_d), np.dot(v3_q,v3_d), np.dot(v4_q,v4_d), np.dot(v5_q,v5_d), np.dot(v6_q,v6_d)
                          ds.cosine(v1_q,v1_d), ds.cosine(v2_q,v2_d), ds.cosine(v3_q,v3_d), ds.cosine(v4_q,v4_d), ds.cosine(v5_q,v5_d), ds.cosine(v6_q,v6_d)]
    return features

features_fasttext = {}
for qid in range(6311):
    for i, features in zip(qmap[qid], features_ft(qid)):
        fasttext_features[str(qid) + "," + str(i)] = features

In [ ]:
#bert
transform_model = SentenceTransformer('bert-large-nli-mean-tokens', device='cuda')
vecs = transform_model.encode(en_titles_arr, batch_size=128)
tr_titles = {}
for did, vec in zip(en_dids, vecs):
    tr_titles[did] = vec

bert1_ft = {}
for qid in range(6311):
    qvecs = transform_model.encode([queries[qid]])[0]
    for i in qmap[qid]:
        if i in tr_titles.keys():
            bert1_ft[str(qid)+","+str(i)] = ds.cosine(qvecs, tr_titles[i])

#roberta
transform_model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens', device='cuda')
vecs = transform_model.encode(en_titles_arr, batch_size=128)
tr_titles = {}
for did, vec in zip(en_dids, vecs):
    tr_titles[did] = vec

bert2_ft = {}
for qid in range(6311):
    qvecs = transform_model.encode([queries[qid]])[0]
    for i in qmap[qid]:
        if i in tr_titles.keys():
            bert2_ft[str(qid)+","+str(i)] = ds.cosine(qvecs, tr_titles[i])

#многоязычный USE
transform_model = SentenceTransformer('distiluse-base-multilingual-cased', device = 'cuda')
vecs = transform_model.encode(titles, batch_size=128)
tr_titles = {}
for did, vec in zip(dids, vecs):
    tr_titles[did] = vec

bert3_ft = {}
for qid in range(6311):
    qvecs = transform_model.encode([queries[qid]])[0]
    for i in qmap[qid]:
        if i in tr_titles.keys():
            bert3_ft[str(qid)+","+str(i)] = ds.cosine(qvecs, tr_titles[i]))


#склеил фичи
bert_features = {}
for i in bert1_ft.keys():
    bert_features[i] = [bert1_ft[i]]
    if i in bert2_ft.keys():
        bert_features[i].append(bert2_ft[i])
    else:
        bert_features[i].append(1)
    if i in bert3_ft.keys():
        bert_features[i].append(bert3_ft[i])
    else:
        bert_features[i].append(1)

In [ ]:
#всякие USE но теперь из tensorflow
qa_m_trans = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')
qa_trans= hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')
trans = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
m_trans = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
lm_trans = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")


def features_use(qid):
    q = [queries[qid]]
    qvecs = qa_m_trans.signatures['question_encoder'](tf.constant(q))
    titles = list(gen_titles(qmap[qid]))
    D = list(gen_docs_qid(qid))
    tvecs = qa_m_trans.signatures['response_encoder'](input = tf.constant(titles), context = tf.constant(D))
    f1 = np.inner(qvecs['outputs'], tvecs['outputs']).flatten().reshape(-1, 1)

    q = [en_queries[qid]]
    titles = list(gen_titles(qmap[qid], english = True))
    D = list(gen_titles(qmap[qid], english = True))
    qvecs = qa_trans.signatures['question_encoder'](tf.constant(q))
    tvecs = qa_trans.signatures['response_encoder'](input = tf.constant(titles), context = tf.constant(D))
    f2 = np.inner(qvecs['outputs'], tvecs['outputs']).flatten().reshape(-1, 1)

    f3 = cosine_similarity(lm_trans(queries[qid]), lm_trans(list(gen_titles(qmap[qid])))).flatten().reshape(-1, 1)
    f4 = cosine_similarity(m_trans(queries[qid]), m_trans(list(gen_titles(qmap[qid])))).flatten().reshape(-1, 1)
    f5 = cosine_similarity(trans([en_queries[qid]])[0].reshape(1,-1), trans(list(gen_titles(qmap[qid], english = True)))).flatten().reshape(-1, 1)
    return np.hstack((f1, f2, f3, f4, f5))

use_features={}
for qid in tqdm.tqdm(range(6311)):
    for did, vec in zip(qmap[qid], features_use(qid)):
        use_features[str(qid)+","+str(did)] = vec

In [ ]:
#соединяем готовые фичи
def get_features(data):
    features = {}
    for i in data:
        f1 = np.zeros(31)
        if str(i[0])+","+str(i[1]) in tfidfbm25_features.keys():
            f1 = tfidfbm25_features[str(i[0])+","+str(i[1])]
        f2 = np.zeros(10)
        if str(i[0])+","+str(i[1]) in doc_tdidfbm25_features.keys():
            f2 = doc_tdidfbm25_features[str(i[0])+","+str(i[1])]
        f3 = np.zeros(12)
        if str(i[0])+","+str(i[1]) in fasttext_features.keys():
            f3 = fasttext_features[str(i[0])+","+str(i[1])]
        f4 = np.zeros(3)
        if str(i[0])+","+str(i[1]) in bert_features.keys():
            f4 = bert_features[str(i[0])+","+str(i[1])]
        f5 = np.zeros(5)
        if str(i[0])+","+str(i[1]) in use_features.keys():
            f5 = use_features[str(i[0])+","+str(i[1])]
        f6 = np.zeros(12)
        if str(i[0])+","+str(i[1]) in docs_features.keys():
            f6 = docs_features[str(i[0])+","+str(i[1])]
        f7 = np.zeros(12)
        if str(i[0])+","+str(i[1]) in hosts_features.keys():
            f7 = hosts_features[str(i[0])+","+str(i[1])]
        f8 = np.zeros(12)
        if str(i[0])+","+str(i[1]) in qd_features.keys():
            f8 = qd_features[str(i[0])+","+str(i[1])]
        f9 = np.zeros(12)
        if str(i[0])+","+str(i[1]) in qh_features.keys():
            f9 = qh_features[str(i[0])+","+str(i[1])]
        features[str(i[0])+","+str(i[1])+","+str(i[2])] = np.hstack((f1, f2, f3, f4, f5, f6, f7, f8, f9))
    return features

In [ ]:
def parse_features(features):
  X =[]
  y = []
  qids =[]
  doc_ids =[]
  for key in features.keys():
    X.append(features[key])
    k = key.split(",")
    y.append(float(k[2]))
    qids.append(int(k[0]))
    doc_ids.append(int(k[1]))
  qids = np.array(qids)
  y = np.array(y)
  X = np.array(X)
  doc_ids= np.array(doc_ids)
  group_counts = np.unique(qids, return_counts=True)[1]
  return X, y, qids, group_counts, doc_ids

X_train, y_train, _, group_counts, _ = parse_features(get_features(train_data))
X_test, _, qids, _, doc_ids = parse_features(get_features(test_data))

In [ ]:
Rank1 = gbm.LGBMRanker(score='ndcg@5', n_estimators=100, num_leaves=127, learning_rate=0.1)
Rank1.fit(X_train, y_train, group=group_counts)
Rank2 = gbm.LGBMRanker(score='ndcg@5', n_estimators=500, num_leaves=127, learning_rate=0.1)
Rank2.fit(X_train, y_train, group=group_counts)
Rank3 = gbm.LGBMRanker(score='ndcg@5', n_estimators=1000, num_leaves=63, learning_rate=0.01)
Rank3.fit(X_train, y_train, group=group_counts)
Rank4 = gbm.LGBMRanker(score='ndcg@5', n_estimators=2000, num_leaves=31, learning_rate=0.01)
Rank4.fit(X_train, y_train, group=group_counts)
Rank5 = gbm.LGBMRanker(score='ndcg@5', n_estimators=4000, num_leaves=31, learning_rate=0.003)
Rank5.fit(X_train, y_train, group=group_counts)
y_pred = Rank1.predict(X_test) + Rank2.predict(X_test) + Rank3.predict(X_test) + Rank4.predict(X_test) + Rank5.predict(X_test)

In [ ]:
out = open('out.txt', 'w')
out.write('QueryId,DocumentId\n')
for qid in np.unique(qids):
  docs_id = copy.deepcopy(doc_ids[np.argwhere(qids == qid).flatten()])
  relevancy = y_pred[np.argwhere(qids == qid).flatten()]
  docs_id = docs_id[np.argsort(relevancy)[::-1]]
  i = 0
  for doc_id in docs_id:
    if i == 5:
      break
    out.write('{},{}\n'.format(qid, doc_id))
    i += 1
out.close()